# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import sys
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torch.cuda import empty_cache
from torchvision import transforms

In [6]:
!git clone https://github.com/cbaguilar/UNETProject.git

Cloning into 'UNETProject'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 65 (delta 19), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (65/65), 743.59 KiB | 8.75 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [7]:

%cd '/content/UNETProject/src'
from models import *
from loss import *
from utils.pre_processing import *
from dataset import *


/content/UNETProject/src


# Utility

In [8]:
class Averagvalue(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy_check(mask, prediction):
    ims = [mask, prediction]
    np_ims = []
    for item in ims:
        if 'PIL' in str(type(item)):
            item = np.array(item)
        elif 'torch' in str(type(item)):
            item = item.cpu().detach().numpy()
        np_ims.append(item)
    compare = np.equal(np.where(np_ims[0] > 0.5, 1, 0), np_ims[1])
    accuracy = np.sum(compare)
    return accuracy / len(np_ims[0].flatten())

In [9]:
def BCELoss(prediction, label):
    masks_probs_flat = prediction.view(-1)
    true_masks_flat = label.float().view(-1)
    loss = nn.BCELoss()(masks_probs_flat, true_masks_flat)
    return loss

In [10]:
def load_hdf5(infile):
    with h5py.File(infile, "r") as f:  # "with" close the file after its nested commands
        return f["image"][()]

In [11]:
def extract_random_patches(full_imgs, full_masks, patch_h, patch_w, N_patches_per_img):

    assert (len(full_imgs.shape) == 4 and len(full_masks.shape) == 4)
    assert (full_imgs.shape[0] == full_masks.shape[0] and full_imgs.shape[2] == full_masks.shape[2] and full_imgs.shape[3] == full_masks.shape[3])
    assert (full_imgs.shape[1] == 1 or full_imgs.shape[1] == 3)
    assert (full_masks.shape[1] == 1)

    N_images, N_channels = full_imgs.shape[0], full_imgs.shape[1]
    img_h, img_w = full_imgs.shape[2], full_imgs.shape[3]
    patches_imgs = np.empty((N_images * N_patches_per_img, N_channels, patch_h, patch_w))
    patches_masks = np.empty((N_images * N_patches_per_img, N_channels, patch_h, patch_w))

    i_patch = 0
    for i in range(N_images):
        k = 0
        while k < N_patches_per_img:
            x_center = np.random.randint(low=int(patch_w / 2), high=img_w - int(patch_w / 2))
            y_center = np.random.randint(low=int(patch_h / 2), high=img_h - int(patch_h / 2))
            patch_img = full_imgs[i, :, y_center - int(patch_h / 2):y_center + int(patch_h / 2), x_center - int(patch_w / 2):x_center + int(patch_w / 2)]
            patch_mask = full_masks[i, :, y_center - int(patch_h / 2):y_center + int(patch_h / 2), x_center - int(patch_w / 2):x_center + int(patch_w / 2)]
            patches_imgs[i_patch] = patch_img
            patches_masks[i_patch] = patch_mask
            i_patch += 1
            k += 1

    return patches_imgs, patches_masks

In [12]:
def get_data_training(train_imgs_original, train_masks, patch_height, patch_width, N_patches_per_image):
    train_imgs = my_PreProc(train_imgs_original)
    train_masks = train_masks / 255.0
    assert (np.min(train_masks) == 0 and np.max(train_masks) == 1)

    patches_imgs_train, patches_masks_train = extract_random_patches(train_imgs, train_masks, patch_height, patch_width, N_patches_per_image)
    return patches_imgs_train, patches_masks_train

# Train and Test Functions

In [ ]:
def test(model, test_loader, epoch):
    model.eval()
    epoch_time = Averagvalue()
    losses = Averagvalue()
    acc = Averagvalue()
    end = time.time()

    with torch.no_grad():
        for i, (image, mask) in enumerate(test_loader):
            image = image.float().to(device).requires_grad_(False)
            mask = mask.to(device).requires_grad_(False)
            pred = model(image)

            loss = loss_fn(pred, mask)
            # loss_fn(pred, onehot(mask, 2).cuda())
            accuracy = accuracy_check(pred, mask)
            acc.update(accuracy, 1)

            losses.update(loss.item(), image.size(0))

        # measure elapsed time
    epoch_time.update(time.time() - end)
    info = 'TEST Epoch: [{0}/{1}]'.format(epoch, N_epochs) + \
           'Test Epoch Time {batch_time.val:.3f} (avg:{batch_time.avg:.3f}) '.format(batch_time=epoch_time) + \
           'Acc {acc.val:f} (avg:{acc.avg:f}) '.format(acc=acc) + \
           'Loss {loss.val:f} (avg:{loss.avg:f}) '.format(loss=losses)
    print(info)
    return losses.avg, acc.avg

In [ ]:
def train(model, train_loader, optimizer, epoch):
    model.train()

    losses = Averagvalue()
    acc = Averagvalue()
    optimizer.zero_grad()
    for i, (image, mask) in enumerate(train_loader):
        image = image.float().to(device)
        mask = mask.float().to(device)
        pred = model(image)

        loss = loss_fn(pred, mask)
        # loss = loss_fn(pred, onehot(mask, 2).cuda())
        accuracy = accuracy_check(pred, mask)
        acc.update(accuracy, 1)

        losses.update(loss.item(), image.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return losses.avg, acc.avg

# Training Code

In [ ]:
def prepare_model(patches_per_image):

  patches_imgs_train, patches_masks_train = get_data_training(
      train_imgs_original=load_hdf5(os.path.join(GOOGLE_DRIVE_PATH, "DRIVE_dataset/DRIVE_imgs_train.hdf5")),
      train_masks=load_hdf5(os.path.join(GOOGLE_DRIVE_PATH, "DRIVE_dataset/DRIVE_groundTruth_train.hdf5")),
      patch_height=48,
      patch_width=48,
      N_patches_per_image=patches_per_image
  )

  patches_imgs_train = np.transpose(patches_imgs_train, (0, 2, 3, 1))
  patches_masks_train = np.transpose(patches_masks_train, (0, 2, 3, 1))
  training_data = vessel_dataset(patches_imgs_train, patches_masks_train, 0.9, split='train')
  validation_data = vessel_dataset(patches_imgs_train, patches_masks_train, 0.9, split='val')

  training_loader = torch.utils.data.DataLoader(training_data, batch_size=4, shuffle=True, num_workers=1)
  validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=1, shuffle=False, num_workers=1)
  return training_loader, validation_loader

In [ ]:
nums = [1, 4, 16, 64, 256, 1024]

device = 'cuda'


for num in nums:
  training_loader, validation_loader = prepare_model(num)
  mymodel = DUNetV1V2(n_channels=1, n_classes=1).to(device)
  optimizer = torch.optim.Adam(mymodel.parameters(), lr=5e-5, weight_decay=1e-5)
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=12)
  # loss_fn = nn.BCELoss()
  loss_fn = DiceLoss(p=1)

  N_epochs = 10

  for epoch in range(N_epochs):
      train_loss, train_acc = train(mymodel, training_loader, optimizer, epoch)
      print(f"num_patches: {num} EPOCH {epoch} | Training Loss: {train_loss}, Training Accuracy: {train_acc}")
      filename = os.path.join(GOOGLE_DRIVE_PATH, 'checkpoints', 'checkpoint_epoch_%03d.pth' % (epoch + 1))
      torch.save({
          'num_patches': num,
          'epoch': epoch + 1,
          'state_dict': mymodel.state_dict(),
          'optimizer': optimizer.state_dict()
      }, filename)
      # val_loss, val_acc = test(mymodel, validation_loader, epoch)
      # scheduler.step(val_loss)
      # print(f"EPOCH {epoch} | Validation Accuracy: {val_acc}")

num_patches: 1 EPOCH 0 | Training Loss: 0.8821908632914225, Training Accuracy: 0.8472873263888889
num_patches: 1 EPOCH 1 | Training Loss: 0.8806025054719713, Training Accuracy: 0.8070529513888889
num_patches: 1 EPOCH 2 | Training Loss: 0.8795546690622965, Training Accuracy: 0.7636935763888888
num_patches: 1 EPOCH 3 | Training Loss: 0.8786631557676527, Training Accuracy: 0.7790798611111109
num_patches: 1 EPOCH 4 | Training Loss: 0.8776960372924805, Training Accuracy: 0.7408637152777777
num_patches: 1 EPOCH 5 | Training Loss: 0.8770704401863946, Training Accuracy: 0.7509765625000001
num_patches: 1 EPOCH 6 | Training Loss: 0.8760446310043335, Training Accuracy: 0.7361328125
num_patches: 1 EPOCH 7 | Training Loss: 0.8746367560492622, Training Accuracy: 0.7060763888888888
num_patches: 1 EPOCH 8 | Training Loss: 0.8743447462717692, Training Accuracy: 0.6087456597222223
num_patches: 1 EPOCH 9 | Training Loss: 0.8731250431802537, Training Accuracy: 0.5693793402777778
num_patches: 4 EPOCH 0 | T

### Evaluation

In [ ]:
# loaded_model = DUNetV1V2(n_channels=1, n_classes=1)
# loaded_model.load_state_dict(torch.load('/content/drive/MyDrive/dunet/checkpoints/DRIVE/checkpoint_epoch_010.pth')['state_dict'])

In [ ]:


device = 'cuda'

loss_funcs = [nn.BCELoss(), DiceLoss(p=1)]

for i in range(2):
  global loss_func
  loss_func = loss_funcs[i]
  num=512
  training_loader, validation_loader = prepare_model(num)
  mymodel = DUNetV1V2(n_channels=1, n_classes=1).to(device)
  optimizer = torch.optim.Adam(mymodel.parameters(), lr=5e-5, weight_decay=1e-5)
  # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=12)
  # loss_fn = nn.BCELoss()
  # loss_fn = DiceLoss(p=1)

  N_epochs = 10

  for epoch in range(N_epochs):
      train_loss, train_acc = train(mymodel, training_loader, optimizer, epoch)
      print(f"loss_func{i} num_patches: {num} EPOCH {epoch} | Training Loss: {train_loss}, Training Accuracy: {train_acc}")
      filename = os.path.join(GOOGLE_DRIVE_PATH, 'checkpoints', 'checkpoint_epoch_%03d.pth' % (epoch + 1))
      torch.save({
          'num_patches': num,
          'epoch': epoch + 1,
          'state_dict': mymodel.state_dict(),
          'optimizer': optimizer.state_dict()
      }, filename)
      # val_loss, val_acc = test(mymodel, validation_loader, epoch)
      # scheduler.step(val_loss)
      # print(f"EPOCH {epoch} | Validation Accuracy: {val_acc}")

loss_func0 num_patches: 512 EPOCH 0 | Training Loss: 0.725548972750807, Training Accuracy: 0.5199940057448392
loss_func0 num_patches: 512 EPOCH 1 | Training Loss: 0.6083346890647792, Training Accuracy: 0.6632574104968415
loss_func0 num_patches: 512 EPOCH 2 | Training Loss: 0.5515872061110308, Training Accuracy: 0.8126370936264231
loss_func0 num_patches: 512 EPOCH 3 | Training Loss: 0.5225636076841814, Training Accuracy: 0.8486565483940973
loss_func0 num_patches: 512 EPOCH 4 | Training Loss: 0.49509189144009724, Training Accuracy: 0.9057488618073641
loss_func0 num_patches: 512 EPOCH 5 | Training Loss: 0.46564590681939283, Training Accuracy: 0.9407444706669573
loss_func0 num_patches: 512 EPOCH 6 | Training Loss: 0.4546026534905347, Training Accuracy: 0.9449512340404402
loss_func0 num_patches: 512 EPOCH 7 | Training Loss: 0.4479232372185733, Training Accuracy: 0.9461669921875012
loss_func0 num_patches: 512 EPOCH 8 | Training Loss: 0.44217337805053425, Training Accuracy: 0.9472185299720276

In [ ]:
test_imgs_original = load_hdf5("/content/drive/MyDrive/dunet/DRIVE_dataset/DRIVE_imgs_test.hdf5")

test_imgs = my_PreProc(test_imgs_original)
test_imgs = np.transpose(test_imgs, (0, 2, 3, 1))

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/content/drive/MyDrive/dunet/DRIVE_dataset/DRIVE_imgs_test.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

### BCELoss

In [ ]:
sample_image = transforms.ToTensor()(test_imgs[3])
mymodel.eval()
with torch.no_grad():
  sample_pred = mymodel(sample_image.float().unsqueeze(0).to(device))
sample_pred = sample_pred.squeeze().detach().cpu().numpy()

f, ax = plt.subplots(1, 2)
ax[0].imshow(sample_image.squeeze(), cmap='gray')
ax[1].imshow(sample_pred, cmap='gray')
plt.show()

### Dice Loss

In [ ]:
sample_image = transforms.ToTensor()(test_imgs[3])
mymodel.eval()
with torch.no_grad():
  sample_pred = mymodel(sample_image.float().unsqueeze(0).to(device))
sample_pred = sample_pred.squeeze().detach().cpu().numpy()

f, ax = plt.subplots(1, 2)
ax[0].imshow(sample_image.squeeze(), cmap='gray')
ax[1].imshow(sample_pred, cmap='gray')
plt.show()

# Lesions Dataset

In [ ]:
patches_imgs_train, patches_masks_train = get_data_training(
    train_imgs_original=load_hdf5(os.path.join(GOOGLE_DRIVE_PATH, "BIGDATA_dataset/BIGDATA_imgs_train.hdf5")),
    train_masks=load_hdf5(os.path.join(GOOGLE_DRIVE_PATH, "BIGDATA_dataset/BIGDATA_groundTruth_train.hdf5")),
    patch_height=64,
    patch_width=64,
    N_patches_per_image=500
)

patches_imgs_train = np.transpose(patches_imgs_train, (0, 2, 3, 1))
patches_masks_train = np.transpose(patches_masks_train, (0, 2, 3, 1))
training_data = vessel_dataset(patches_imgs_train, patches_masks_train, 0.9, split='train')
validation_data = vessel_dataset(patches_imgs_train, patches_masks_train, 0.9, split='val')

training_loader = torch.utils.data.DataLoader(training_data, batch_size=4, shuffle=True, num_workers=1)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
device = 'cuda'

mymodel = DUNetV1V2(n_channels=1, n_classes=1).to(device)
optimizer = torch.optim.Adam(mymodel.parameters(), lr=5e-5, weight_decay=1e-5)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=12)
loss_fn = nn.BCELoss()
# loss_fn = DiceLoss(p=1)

N_epochs = 10

In [ ]:
for epoch in range(N_epochs):
    train_loss, train_acc = train(mymodel, training_loader, optimizer, epoch)
    print(f"EPOCH {epoch} | Training Loss: {train_loss}, Training Accuracy: {train_acc}")
    filename = os.path.join(GOOGLE_DRIVE_PATH, 'checkpoints', 'checkpoint_epoch_%03d.pth' % (epoch + 1))
    torch.save({
        'epoch': epoch + 1,
        'state_dict': mymodel.state_dict(),
        'optimizer': optimizer.state_dict()
    }, filename)
    # val_loss, val_acc = test(mymodel, validation_loader, epoch)
    # scheduler.step(val_loss)
    # print(f"EPOCH {epoch} | Validation Accuracy: {val_acc}")

In [ ]:
loaded_model = DUNetV1V2(n_channels=1, n_classes=1)
loaded_model.load_state_dict(torch.load('/content/drive/MyDrive/dunet/checkpoints/BIGDATA/checkpoint_epoch_010.pth')['state_dict'])

In [ ]:
test_imgs_original = load_hdf5("/content/drive/MyDrive/dunet/BIGDATA_dataset/BIGDATA_imgs_test.hdf5")

test_imgs = my_PreProc(test_imgs_original)
test_imgs = np.transpose(test_imgs, (0, 2, 3, 1))

In [ ]:
sample_image = transforms.ToTensor()(test_imgs[10])
loaded_model.eval()
with torch.no_grad():
  sample_pred = loaded_model(sample_image.float().unsqueeze(0))
sample_pred = sample_pred.squeeze().detach().cpu().numpy()

f, ax = plt.subplots(1, 2)
ax[0].imshow(sample_image.squeeze(), cmap='gray')
ax[1].imshow(sample_pred, cmap='gray')
plt.show()